In [1]:
using CRCBS
using LightGraphs, MetaGraphs
G = initialize_full_grid_graph()
starts = [1,5,8]
goals = [4,1,2]
lambda = 0.5
epsilon = 0.1
t_delay = 0.5
mapf = MAPF(G,starts,goals)


┌ Info: Recompiling stale cache file /home/peltzer/.julia/compiled/v1.0/CRCBS/26Ksw.ji for CRCBS [edef75b8-3ac4-11e9-0392-bf149c536ee3]
└ @ Base loading.jl:1190


MAPF{MetaGraph{Int64,Float64}}({112, 196} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0), [1, 5, 8], [4, 1, 2])

In [2]:
CBS(mapf)

Optimal Solution Found! Cost = 15


(Array{LightGraphs.SimpleGraphs.SimpleEdge{Int64},1}[[Edge 1 => 2, Edge 2 => 13, Edge 13 => 14, Edge 14 => 15, Edge 15 => 4], [Edge 5 => 4, Edge 4 => 3, Edge 3 => 2, Edge 2 => 1], [Edge 8 => 7, Edge 7 => 6, Edge 6 => 5, Edge 5 => 4, Edge 4 => 3, Edge 3 => 2]], 15)

In [1]:
mg = MetaDiGraph()
add_vertex!(mg,Dict(:x=>1,:n_delay=>1.0))
add_vertex!(mg,Dict(:x=>2,:n_delay=>1.0))
add_edge!(mg,1,2)
set_prop!(mg,Edge(1,2),:foo,5)

UndefVarError: UndefVarError: MetaDiGraph not defined

In [5]:
for v in vertices(mg)
    print(v)
end

12

In [6]:
Edge(1,2).src

1

In [12]:
get_prop(mg,Edge(2,1),:foo)

5

In [3]:
using CBS

ArgumentError: ArgumentError: Package CBS not found in current path:
- Run `import Pkg; Pkg.add("CBS")` to install the CBS package.
